In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.optimizers import Adam

import pickle
from sklearn.model_selection import train_test_split 
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import os
import csv
from PIL import Image


Using TensorFlow backend.


In [2]:
(train_set, train_label_raw), (X_test, y_test) = mnist.load_data()
train_label = to_categorical(train_label_raw)
train_data = np.asarray(train_set).reshape(60000, 28, 28, 1)

In [3]:
gen_images_raw = pd.read_csv('generated_images_47600_0.923000.csv', header=None)
X_fake_raw = np.asarray(gen_images_raw).reshape(100000, 28, 28, 1)

gen_labels_raw = pd.read_csv('generated_labels_47600_0.923000.csv', header=None)
y_fake_raw = to_categorical(gen_labels_raw)
X_fake, y_fake = shuffle(X_fake_raw, y_fake_raw)

In [4]:
class classifier():
    def __init__(self):
        
        self.model = Sequential()
        # model.add(Conv2D(5, kernel_size=3, activation='relu', input_shape=(28,28,1)))
        self.model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
        self.model.add(MaxPooling2D(pool_size=(4, 4)))
        self.model.add(Conv2D(32, kernel_size=3, activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Flatten())
        self.model.add(Dense(10, activation='softmax'))
        
    def compile(self):
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def train(self, totTrain=100_000, epochs=10, ratio=(1,1)): #Real to Fake
        sum = ratio[0] + ratio[1]
        realTrain = ( totTrain // sum ) * ratio[0] 
        fakeTrain = ( totTrain // sum ) * ratio[1] 

        X_train_raw, X_val, y_train_raw, y_val = train_test_split(train_data, train_label, test_size=0.33, random_state=43)

        X_train = np.concatenate((X_train_raw[:realTrain], X_fake[:fakeTrain]))
        y_train = np.concatenate((y_train_raw[:realTrain], y_fake[:fakeTrain]))
        
        print(X_train.shape)
        print(X_train_raw[:realTrain].shape)
        print(X_fake[:fakeTrain].shape)

        # model.summary()

        self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs)
        os.makedirs('classif_history/', exist_ok=True)

        root = 'classif_history/'
        with open(root + 'classif_'+ str(totTrain) + '_' + str(ratio[0]) + '_' + str(ratio[1]) + '.csv', 'w') as f:
            for key in self.model.history.history.keys():
                f.write("%s,%s\n"%(key,self.model.history.history[key]))

        X_test_reshaped = X_test.reshape(10000, 28, 28, 1)
        loss, acc = self.model.evaluate(X_test_reshaped, to_categorical(y_test))
        print(acc)
        with open(root + 'classif_acc.csv', 'a') as b:
            b.write(str(totTrain) + ',' + str(ratio[0]) + ',' + str(ratio[1]) + ',' + "%f\n"%(acc))
            

In [5]:
ratio_array = [ (0,1), (1,0), (1,1), (2,1), (1,2), (3,1), (1,3), (5,1), (1,5), (10,1), (1,10)]
train_array = [ 1000 ]




for num in train_array:
    for ratio in ratio_array:
        
#         del cl
        if (ratio != ratio_array[0]) & (num != train_array[0]):
            del cl
        cl = classifier()
        cl.compile()


#         cl.reset_weights()
        cl.train(totTrain=num, epochs=10, ratio=ratio)


(1000, 28, 28, 1)
(0, 28, 28, 1)
(1000, 28, 28, 1)
Train on 1000 samples, validate on 19800 samples
Epoch 1/10
1000/1000 [==============================] - 4s 4ms/step - loss: 11.5106 - acc: 0.2300 - val_loss: 10.4529 - val_acc: 0.2842
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 9.3516 - acc: 0.3740 - val_loss: 8.7656 - val_acc: 0.4154
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 8.5236 - acc: 0.4440 - val_loss: 8.4299 - val_acc: 0.4503
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 8.1066 - acc: 0.4650 - val_loss: 7.4723 - val_acc: 0.4736
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1195 - acc: 0.5460 - val_loss: 5.7060 - val_acc: 0.5849
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.3809 - acc: 0.6170 - val_loss: 5.5661 - val_acc: 0.6012
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.0208 - acc: 0.64

Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 13.0718 - acc: 0.1890 - val_loss: 12.9632 - val_acc: 0.1951
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 13.0718 - acc: 0.1890 - val_loss: 12.9632 - val_acc: 0.1951
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 13.0718 - acc: 0.1890 - val_loss: 12.9632 - val_acc: 0.1951
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 13.0718 - acc: 0.1890 - val_loss: 12.9632 - val_acc: 0.1951
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 13.0718 - acc: 0.1890 - val_loss: 12.9632 - val_acc: 0.1951
Epoch 10/10
10000/10000 [==============================] - 1s 61us/step
0.1938
(1000, 28, 28, 1)
(250, 28, 28, 1)
(750, 28, 28, 1)
Train on 1000 samples, validate on 19800 samples
Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 14.4902 - acc: 0.1010 - val_loss: 14.5079 - val_acc: 0.0999
Epoc

990/990 [==============================] - 2s 2ms/step - loss: 14.3598 - acc: 0.1091 - val_loss: 14.5169 - val_acc: 0.0993
Epoch 10/10
10000/10000 [==============================] - 1s 85us/step
0.101


In [6]:
ratio_array = [ (1,2) ]
train_array = [ 150_000 ]



for num in train_array:
    for ratio in ratio_array:
        del cl
#         if (ratio != ratio_array[0]) & (num != train_array[0]):
#             del cl
        cl = classifier()
        cl.train(totTrain=num, epochs=25, ratio=ratio)


(140200, 28, 28, 1)
(40200, 28, 28, 1)
(100000, 28, 28, 1)


RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.